In [ ]:
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import nd2reader
import skimage
import numba
from tqdm.auto import tqdm
import timeit

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext pyinstrument

In [ ]:
from paulssonlab.image_analysis import blur

For discussions of algorithms, see:
- https://stackoverflow.com/questions/98359/fastest-gaussian-blur-implementation
- https://github.com/bfraboni/FastGaussianBlur
- http://blog.ivank.net/fastest-gaussian-blur.html
- https://dsp.stackexchange.com/questions/50576/fastest-available-algorithm-to-blur-an-image-low-pass-filter
- https://github.com/pelson/antigrain/blob/64c9125e2b350a422c08d7fa8fff023400ad3f9f/agg-2.4/include/agg_blur.h

# Setup

In [ ]:
blur_funcs = ["ndi.gaussian_filter", "blur.scipy_box_blur_2d", "blur.scipy_box_blur", "blur.numba_stack_blur"]
#sigmas = np.concatenate((np.linspace(0.1, 3, 5), np.linspace(4, 50, 5)))
#sigmas = [1,3,5,10,25,50]
sigmas = np.concatenate((np.arange(1, 10), np.arange(10, 50, 5)))
kwargs = {"mode": "nearest"}

In [ ]:
nd2 = nd2reader.ND2Reader("/home/jqs1/scratch/jqs1/microscopy/220704/220704rbs_library_fish.nd2")
img = nd2.get_frame_2D(v=50, t=20, c=2)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(img)

# Benchmarks

In [ ]:
%%time
rows = []
for sigma in tqdm(sigmas):
    ref = None
    for func_name in tqdm(blur_funcs, leave=False):
        eval_str = f"{func_name}(img2, sigma, **kwargs)"
        img2 = img.copy()
        blurred = eval(eval_str, globals())
        t = timeit.Timer(eval_str, setup="img2 = img.copy()", globals=globals()).timeit(10)
        if ref is None:
            ref = blurred
            rms = 0
        else:
            rms = np.sqrt(((blurred - ref)**2).sum())
        rows.append((func_name, sigma, t, rms))
df = pd.DataFrame(rows, columns=["function", "sigma", "t", "rms"])

In [ ]:
df.hvplot.scatter("sigma", "t", by="function")

# Performance

# Accuracy matrix

# Auxiliary benchmarks

# Test

In [ ]:
%%pyinstrument
ndi.gaussian_filter(img, 1)

In [ ]:
%%pyinstrument
img2 = img.copy()
for i in range(5):
    #ndi.gaussian_filter(img, 1)
    blur.scipy_box_blur(img, 1, output=img2)

In [ ]:
timeit.Timer("sb(img2, 10)", setup="img2 = img.copy()", globals=globals()).timeit(100)

In [ ]:
timeit.Timer("blur.scipy_box_blur_2d(img, 10)", globals=globals()).timeit(5)

In [ ]:
timeit.Timer("blur.scipy_box_blur(img, 10)", globals=globals()).timeit(5)

In [ ]:
timeit.Timer("blur.scipy_box_blur(img, 10)", globals=globals()).timeit(5)